In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import requests
import random
from ip_pool import ip_pool
import time
import multiprocessing


def get_abstract(url):
    proxies = get_ip(url=url)
    # 发送 GET 请求并获取网页内容
    response = requests.get(url, proxies=proxies)
    # 使用 BeautifulSoup 解析网页内容
    soup = BeautifulSoup(response.text, 'html.parser')
    # 找到具有 class 为 "black" 的所有标签
    tag = soup.find(class_="abstractSection abstractInFull")
    try:
        content = tag.get_text()
    except:
        content = 'Delete'
    print(content)
    print()
    return content


def get_ip(url):
    while True:
        ip = random.choice(ip_pool)
        proxies = {
            'http': 'http://' + ip,
            'https': 'https://' + ip
        }
        page = requests.get(url=url, proxies=proxies, timeout=(3, 3))
        if page.status_code == 200:
            print(ip)
            return proxies
        else:
            print('无效IP')


def loop_for_get_abstract(url):
    while True:
        try:
            file.loc[i, 'abstract'] = get_abstract(url)
            break  # 如果执行成功，退出循环
        except Exception as e:
            print('Error:', e)


file_path = r'./output/CHI_with_abstract/CHI_process_2023.csv'
file = pd.read_csv(file_path)

for i in range(len(file)):

    # 从上次之后开始写
    if not pd.isna(file.loc[i, 'abstract']):
        continue

    if i % 20 == 0:
        file.to_csv(file_path, index=False, encoding='utf-8-sig')

    print(f'>>>>>>>>>>>>>>>>>>    Paper {i}     <<<<<<<<<<<<<<<<<<<<')
    print()
    url = file.loc[i, 'web']

    # 创建子进程，并传入loop_for_get_abstract函数
    process = multiprocessing.Process(target=loop_for_get_abstract(url=url))
    # 启动子进程
    process.start()
    # 等待子进程结束，最多等待30秒
    process.join(timeout=30)

    # 如果子进程仍在运行，则终止子进程
    if process.is_alive():
        process.terminate()
        # 等待子进程结束
        process.join()

    # 主进程继续执行其他代码

file.to_csv(file_path, index=False, encoding='utf-8-sig')

In [4]:
# 一直error